In [1]:
import pandas as pd
from pycaret import classification
from pycaret.classification import *

In [9]:
df = pd.read_csv(r"C:\Users\Matheus\Desktop\Estudo\Kafka\Transação_Credito\Previsao\Dados\card_transdata.csv")

In [4]:
df.head(5)

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   distance_from_home              1000000 non-null  float64
 1   distance_from_last_transaction  1000000 non-null  float64
 2   ratio_to_median_purchase_price  1000000 non-null  float64
 3   repeat_retailer                 1000000 non-null  float64
 4   used_chip                       1000000 non-null  float64
 5   used_pin_number                 1000000 non-null  float64
 6   online_order                    1000000 non-null  float64
 7   fraud                           1000000 non-null  float64
dtypes: float64(8)
memory usage: 61.0 MB


In [13]:
#Verificando se a base de dados esta desbalanceada
positivos = df[df['fraud'] == 1].shape[0]
negativos = df[df['fraud'] == 0].shape[0]

perc = (positivos/(positivos + negativos))*100

print(f"Quantidade de diagnósticos positivos: {positivos} "  )
print(f"Quantidade de diagnósticos negativos: {negativos} "  )
print(f"Percentual de positivos: {perc}%")

Quantidade de diagnósticos positivos: 87403 
Quantidade de diagnósticos negativos: 912597 
Percentual de positivos: 8.7403%


In [14]:
categorias =['repeat_retailer', 'used_chip', 'used_pin_number','online_order']

In [15]:
inicialização = setup(df, target='fraud', normalize=True, fix_imbalance=True,
                      categorical_features= categorias)

,Description,Value
0,Session id,6672
1,Target,fraud
2,Target type,Binary
3,Original data shape,"(1000000, 8)"
4,Transformed data shape,"(1577636, 8)"
5,Transformed train set shape,"(1277636, 8)"
6,Transformed test set shape,"(300000, 8)"
7,Numeric features,3
8,Categorical features,4
9,Preprocess,True


In [16]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,1.0000,0.9999,0.9999,0.9998,0.9999,0.9998,0.9998,1.9560
rf,Random Forest Classifier,1.0000,1.0000,0.9999,1.0000,0.9999,0.9999,0.9999,29.0470
ada,Ada Boost Classifier,0.9998,1.0000,0.9990,0.9986,0.9988,0.9987,0.9987,12.0150
et,Extra Trees Classifier,0.9995,1.0000,0.9979,0.9967,0.9973,0.9970,0.9970,26.2200
gbc,Gradient Boosting Classifier,0.9992,1.0000,0.9965,0.9940,0.9952,0.9948,0.9948,45.6130
knn,K Neighbors Classifier,0.9984,0.9995,0.9966,0.9850,0.9907,0.9898,0.9899,9.1730
lightgbm,Light Gradient Boosting Machine,0.9980,1.0000,0.9969,0.9807,0.9888,0.9877,0.9877,2.9990
lr,Logistic Regression,0.9348,0.9792,0.9476,0.5773,0.7175,0.6831,0.7099,2.1390
svm,SVM - Linear Kernel,0.9328,0.9788,0.9625,0.5684,0.7147,0.6795,0.7099,1.4820
dummy,Dummy Classifier,0.9126,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.9820


In [17]:
model = create_model(best_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0000,1.0000,0.9997,0.9998,0.9998,0.9998
1,1.0000,0.9999,0.9998,1.0000,0.9999,0.9999,0.9999
2,1.0000,1.0000,1.0000,0.9997,0.9998,0.9998,0.9998
3,1.0000,1.0000,1.0000,0.9997,0.9998,0.9998,0.9998
4,1.0000,0.9999,0.9998,0.9997,0.9998,0.9997,0.9997
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,1.0000,1.0000,0.9998,0.9999,0.9999,0.9999
7,0.9999,0.9998,0.9997,0.9997,0.9997,0.9996,0.9996
8,0.9999,0.9999,0.9998,0.9992,0.9995,0.9995,0.9995


In [18]:
evaluate_model(model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [29]:
final_best = finalize_model(model)
save_model(final_best, '../models/fraud_detection')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['distance_from_home',
                                              'distance_from_last_transaction',
                                              'ratio_to_median_purchase_price'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean'))),
                 ('categorical_imput...
                                     transformer=StandardScaler(copy=True,
                                

In [24]:
df[df['fraud'] == 1]

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
13,2.131956,56.372401,6.358667,1.0,0.0,0.0,1.0,1.0
24,3.803057,67.241081,1.872950,1.0,0.0,0.0,1.0,1.0
29,15.694986,175.989182,0.855623,1.0,0.0,0.0,1.0,1.0
35,26.711462,1.552008,4.603601,1.0,1.0,0.0,1.0,1.0
36,10.664474,1.565769,4.886521,1.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...
999908,45.296658,0.882736,8.856861,1.0,0.0,0.0,1.0,1.0
999916,167.139756,0.282253,0.308468,1.0,0.0,0.0,1.0,1.0
999919,124.640118,0.004416,0.434885,1.0,0.0,0.0,1.0,1.0
999939,51.412900,3.429330,29.914254,1.0,0.0,0.0,1.0,1.0


In [3]:
loaded_model = load_model('../models/fraud_detection')

novos_dados = pd.DataFrame({
    'distance_from_home': [26.711462],
    'distance_from_last_transaction': [56.372401]	,
    'ratio_to_median_purchase_price': [4.603601],
    'repeat_retailer': [0],
    'used_chip': [0],
    'used_pin_number': [0],
    'online_order': [1],
})
predict_model(loaded_model, raw_score= True, data = novos_dados)

Transformation Pipeline and Model Successfully Loaded


,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,prediction_label,prediction_score_0,prediction_score_1
0,26.711462,56.372402,4.603601,0,0,0,1,1,0.0,1.0
